# Feature Engineering

### Caricamento dei file train_data_labeled.pkl 

In [4]:
import os
import pandas as pd

path_dataframes = "dataset/dataframes"
file_path = os.path.join(path_dataframes, "train_data_labeled.pkl")

# Carica il DataFrame usando pd.read_pickle
df_train_labeled = pd.read_pickle(file_path)

Stampo info sul dataset

In [5]:
print("Contenuto del DataFrame:")
print(df_train_labeled.head())

print("\nDimensione del DataFrame:")
print(df_train_labeled.shape)

Contenuto del DataFrame:
    TIME   P1   P2   P3   P4   P5   P6   P7  Case  Spacecraft  ...  SV3  SV4  \
0  0.000  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
1  0.001  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
2  0.002  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
3  0.003  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   
4  0.004  2.0  2.0  2.0  2.0  2.0  2.0  2.0     1           1  ...  100  100   

   BP1  BP2  BP3 BP4 BP5 BP6 BP7 BV1  
0   No   No   No  No  No  No  No  No  
1   No   No   No  No  No  No  No  No  
2   No   No   No  No  No  No  No  No  
3   No   No   No  No  No  No  No  No  
4   No   No   No  No  No  No  No  No  

[5 rows x 23 columns]

Dimensione del DataFrame:
(212577, 23)


Funzione che calcola le metriche nel dominio del tempo:

In [6]:
import numpy as np

def time_domain_metrics(signal):
    """
    Calcola le metriche nel dominio del tempo per un array 1D (signal).
    Restituisce un dizionario con i valori calcolati.
    """
    metrics = {}
    metrics["mean"] = np.mean(signal)
    metrics["median"] = np.median(signal)
    metrics["p25"] = np.percentile(signal, 25)
    metrics["p75"] = np.percentile(signal, 75)
    metrics["variance"] = np.var(signal)
    metrics["line_integral"] = np.trapezoid(signal)  # approssima l'integrale
    metrics["min"] = np.min(signal)
    metrics["max"] = np.max(signal)
    return metrics

Funzione che calcola le metriche nel dominio della frequenza (FFT)

In [7]:
def frequency_domain_metrics(signal, sampling_rate=1000):
    """
    Calcola alcune metriche spettrali (fft) per un array 1D (signal).
    Restituisce un dizionario con i valori calcolati.
    Puoi estendere questa funzione con SNR, SINAD, ecc.
    """
    fft_result = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal), d=1/sampling_rate)
    
    # Teniamo solo le frequenze positive (>0), escludendo la componente DC
    positive_indices = np.where(freqs > 0)
    fft_result = fft_result[positive_indices]
    freqs = freqs[positive_indices]
    
    power_spectrum = np.abs(fft_result) ** 2
    
    # Esempio di metriche spettrali basilari
    metrics = {}
    metrics["peak_value"] = np.max(power_spectrum)
    metrics["peak_freq"] = freqs[np.argmax(power_spectrum)]
    metrics["sum_power_spectrum"] = np.sum(power_spectrum)
    metrics["std_power_spectrum"] = np.std(power_spectrum)
    
    # (Facoltativo) Esempio di RMS nel dominio frequenziale
    metrics["rms_freq"] = np.sqrt(np.mean(power_spectrum))
    
    return metrics

Funzione per aggregare un singolo Case

In [8]:
def aggregate_case(case_group, sampling_rate=1000):
    """
    case_group: subset del DataFrame per un singolo Case
    Ritorna un dizionario con le metriche calcolate per P1..P7.
    """
    result = {}
    
    # Per le colonne che rimangono invariate all'interno dello stesso Case,
    # prendiamo il valore dalla prima riga (o dall'ultima, è uguale).
    # Aggiungile a piacimento (Spacecraft, Condition, ecc.)
    columns_to_keep = ["Spacecraft", "Condition",
                       "SV1", "SV2", "SV3", "SV4", "BP1", "BP2", "BP3", 
                       "BP4", "BP5", "BP6", "BP7", "BV1"]
    for col in columns_to_keep:
        if col in case_group.columns:
            result[col] = case_group[col].iloc[0]
    
    # Per ogni colonna di segnale (P1..P7) calcoliamo le metriche
    signal_columns = [col for col in case_group.columns 
                      if col.startswith("P") and col not in columns_to_keep]
    
    for col in signal_columns:
        signal = case_group[col].values
        
        # Calcolo metriche dominio del tempo
        td_metrics = time_domain_metrics(signal)
        # Calcolo metriche dominio della frequenza
        fd_metrics = frequency_domain_metrics(signal, sampling_rate=sampling_rate)
        
        # Inseriamo le metriche nel dizionario finale con un prefisso
        for k, v in td_metrics.items():
            result[f"{col}_time_{k}"] = v
        for k, v in fd_metrics.items():
            result[f"{col}_freq_{k}"] = v
    
    return result

Funzione per splittare i dati in 3 finestre temporali, per poi aggregare per ogni split, triplicando così i dati di training

In [9]:
import pandas as pd

def split_and_aggregate(df, sampling_rate=1000):
    """
    Suddivide ogni Case in 3 finestre temporali e calcola le metriche per ogni finestra.
    """
    window_size = 0.4  # 400ms
    windows = [(0, 0.4), (0.4, 0.8), (0.8, 1.2)]
    
    aggregated_data = []

    for case, case_group in df.groupby("Case"):
        for i, (start, end) in enumerate(windows):
            # Filtra la finestra temporale
            window_data = case_group[(case_group["TIME"] >= start) & (case_group["TIME"] < end)]
            
            if not window_data.empty:
                # Aggrega le metriche per questa finestra
                agg_result = aggregate_case(window_data, sampling_rate)
                agg_result["Case"] = case
                agg_result["Window_ID"] = i + 1  # Identificatore della finestra
                aggregated_data.append(agg_result)

    return pd.DataFrame(aggregated_data)

Creo il nuovo dataframe di training triplicato

In [10]:
# Applica la funzione di aggregazione con finestre
df_aggregated = split_and_aggregate(df_train_labeled)

# Mostra il risultato con print
print(df_aggregated.shape)
print(df_aggregated.head())

# Controllo del bilanciamento delle finestre temporali
print(df_aggregated['Window_ID'].value_counts())

(531, 107)
   Spacecraft Condition  SV1  SV2  SV3  SV4 BP1 BP2 BP3 BP4  ...  \
0           1    Normal  100  100  100  100  No  No  No  No  ...   
1           1    Normal  100  100  100  100  No  No  No  No  ...   
2           1    Normal  100  100  100  100  No  No  No  No  ...   
3           1    Normal  100  100  100  100  No  No  No  No  ...   
4           1    Normal  100  100  100  100  No  No  No  No  ...   

  P7_time_line_integral P7_time_min P7_time_max P7_freq_peak_value  \
0            784.906580    0.017144    5.013518        5020.028779   
1            787.373792   -0.002641    5.016796        5610.686996   
2            789.525599   -0.003006    5.017115        5325.708422   
3            785.214600   -0.002262    4.994830        5071.356039   
4            787.175441   -0.003732    4.999298        5609.065086   

   P7_freq_peak_freq  P7_freq_sum_power_spectrum  P7_freq_std_power_spectrum  \
0          65.000000                26930.154869                  501.116403   

Esporto il df triplicato in csv e in pickle nella cartella dataframes

In [ ]:
# Definisci il percorso della cartella di destinazione
output_dir = "dataset/dataframes"
os.makedirs(output_dir, exist_ok=True)  # Crea la cartella se non esiste

# Definisci il percorso completo per il file CSV
output_path = os.path.join(output_dir, "train_data_aggregated_split.csv")
output_path_pkl = os.path.join(output_dir, "train_data_aggregated_split.pkl")

# Esporta il DataFrame in CSV
df_aggregated.to_csv(output_path, index=False)
df_aggregated.to_pickle(output_path_pkl)

print(f"Dati aggregati esportati in: {output_path}")


Dati aggregati esportati in: dataset/dataframes\train_data_aggregated_split.csv


## Facciamo la stessa cosa per il test

Prendo il dataset di test con le 'label'

In [12]:
path_dataframes = "dataset/dataframes"
file_path = os.path.join(path_dataframes, "test_data_labeled.pkl")

# Carica il DataFrame usando pd.read_pickle
df_test_labeled = pd.read_pickle(file_path)

Aggreghiamo e split

In [13]:
# Applica la funzione di aggregazione con finestre
df_test_aggregated = split_and_aggregate(df_test_labeled)

# Mostra il risultato con print
print(df_test_aggregated.shape)
print(df_test_aggregated.head())

# Controllo del bilanciamento delle finestre temporali
print(df_test_aggregated['Window_ID'].value_counts())

(138, 94)
   Spacecraft  P1_time_mean  P1_time_median  P1_time_p25  P1_time_p75  \
0           1      1.984074        1.968528     1.897964     2.057444   
1           1      1.984509        1.963438     1.898697     2.057745   
2           1      1.984681        1.963766     1.899070     2.057667   
3           1      1.984472        1.960340     1.892148     2.062852   
4           1      1.985048        1.958879     1.894102     2.063106   

   P1_time_variance  P1_time_line_integral  P1_time_min  P1_time_max  \
0          0.089958             791.609870     0.599879     4.409431   
1          0.090624             791.774258     0.561861     4.411653   
2          0.090409             793.837928     0.561754     4.411926   
3          0.093266             791.761654     0.479821     4.100935   
4          0.093737             791.969210     0.453900     4.103057   

   P1_freq_peak_value  ...  P7_time_line_integral  P7_time_min  P7_time_max  \
0         1187.765081  ...             

Esporto il df di test triplicato

In [14]:
# Definisci il percorso della cartella di destinazione
output_dir = "dataset/dataframes"
os.makedirs(output_dir, exist_ok=True)  # Crea la cartella se non esiste

# Definisci il percorso completo per il file CSV
output_path = os.path.join(output_dir, "test_data_aggregated_split.csv")
output_path_pkl = os.path.join(output_dir, "test_data_aggregated_split.pkl")

# Esporta il DataFrame in CSV
df_test_aggregated.to_csv(output_path, index=False)
df_test_aggregated.to_pickle(output_path_pkl)

print(f"Dati aggregati esportati in: {output_path}")


Dati aggregati esportati in: dataset/dataframes\test_data_aggregated_split.csv
